In [1]:
import numpy as np
from itertools import combinations

In [2]:
test = ['19, 13, 30 @ -2,  1, -2',
        '18, 19, 22 @ -1, -1, -2',
        '20, 25, 34 @ -2, -2, -4',
        '12, 31, 28 @ -1, -2, -1',
        '20, 19, 15 @  1, -5, -3']

data = np.genfromtxt('day24_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
def get_hail(data):
    hail = []
    for line in data:
        poses, vels = line.split(' @ ')
        pos = poses.split(', ')
        vel = vels.split(', ')
        for i in range(0, len(pos)):
            pos[i] = np.int64(pos[i])
            vel[i] = np.int64(vel[i])
        hail.append([pos, vel])
    return np.array(hail)

def xy_intersection(hail, low, high):
    in_region = 0
    
    for i, j in combinations(np.arange(0, len(hail)), 2):
        dydx_i = hail[i][1,1]/hail[i][1,0]
        dydx_j = hail[j][1,1]/hail[j][1,0]
        
        #check if parallel
        if dydx_i == dydx_j:
            continue
            
        c_i = hail[i][0,1] - (dydx_i * hail[i][0,0])
        c_j = hail[j][0,1] - (dydx_j * hail[j][0,0])
        
        Px = (c_j - c_i)/(dydx_i - dydx_j)
        if Px < low or Px > high:
            continue
        elif (Px < hail[i][0,0] and hail[i][1,0] > 0) or\
             (Px > hail[i][0,0] and hail[i][1,0] < 0):
            continue
        elif (Px < hail[j][0,0] and hail[j][1,0] > 0) or\
             (Px > hail[j][0,0] and hail[j][1,0] < 0):
            continue
            
        Py = (dydx_i*Px) + c_i
        if Py < low or Py > high:
            continue
        elif (Py < hail[i][0,1] and hail[i][1,1] > 0) or\
             (Py > hail[i][0,1] and hail[i][1,1] < 0):
            continue
        elif (Py < hail[j][0,1] and hail[j][1,1] > 0) or\
             (Py > hail[j][0,1] and hail[j][1,1] < 0):
            continue
        
        
        in_region += 1
        
    return in_region

def part1(data, low, high):
    hail = get_hail(data)
    return xy_intersection(hail, low, high)

print(part1(test, 7, 27))
print('Part 1 result:', part1(data, 200000000000000, 400000000000000))

2
Part 1 result: 20336


In [4]:
def intersect(hail_i, hail_j, vel):
    dydx_i = np.float128(hail_i[1,1]-vel[1])/np.float128(hail_i[1,0]-vel[0])
    dydx_j = np.float128(hail_j[1,1]-vel[1])/np.float128(hail_j[1,0]-vel[0])
    if dydx_i == dydx_j:
        return (np.nan, np.nan), (np.nan, np.nan)
    
    c_i = hail_i[0,1] - (dydx_i * hail_i[0,0])
    c_j = hail_j[0,1] - (dydx_j * hail_j[0,0])
    
    Px = (c_j - c_i)/(dydx_i - dydx_j)
    
    if (Px < hail_i[0,0] and (hail_i[1,0]-vel[0]) > 0) or\
       (Px > hail_i[0,0] and (hail_i[1,0]-vel[0]) < 0):
        return (np.nan, np.nan), (np.nan, np.nan)
    elif (Px < hail_j[0,0] and (hail_j[1,0]-vel[0]) > 0) or\
         (Px > hail_j[0,0] and (hail_j[1,0]-vel[0]) < 0):
        return (np.nan, np.nan), (np.nan, np.nan)
    
    Py = (dydx_i*Px) + c_i
    if (Py < hail_i[0,1] and (hail_i[1,1]-vel[1]) > 0) or\
       (Py > hail_i[0,1] and (hail_i[1,1]-vel[1]) < 0):
        return (np.nan, np.nan), (np.nan, np.nan)
    elif (Py < hail_j[0,1] and (hail_j[1,1]-vel[1]) > 0) or\
         (Py > hail_j[0,1] and (hail_j[1,1]-vel[1]) < 0):
        return (np.nan, np.nan), (np.nan, np.nan)
    
    t_i = (Px-hail_i[0,0])/(hail_i[1,0]-vel[0])
    t_j = (Px-hail_j[0,0])/(hail_j[1,0]-vel[0])
    
    return (np.rint(Px), np.rint(Py)), (t_i, t_j)

def possible_velocities(hail):
    not_x = []
    not_y = []
    not_z = []
    
    for i, j in combinations(np.arange(0, len(hail)), 2):
        if (hail[i][0,0] > hail[j][0,0] and hail[i][1,0] > hail[j][1,0]) or\
           (hail[i][0,0] < hail[j][0,0] and hail[i][1,0] < hail[j][1,0]):
            for x in range(np.min([hail[i][1,0], hail[j][1,0]]), np.max([hail[i][1,0], hail[j][1,0]])):
                not_x.append(x)
                
        if (hail[i][0,1] > hail[j][0,1] and hail[i][1,1] > hail[j][1,1]) or\
           (hail[i][0,1] < hail[j][0,1] and hail[i][1,1] < hail[j][1,1]):
            for y in range(np.min([hail[i][1,1], hail[j][1,1]]), np.max([hail[i][1,1], hail[j][1,1]])):
                not_y.append(y)
                
        if (hail[i][0,2] > hail[j][0,2] and hail[i][1,2] > hail[j][1,2]) or\
           (hail[i][0,2] < hail[j][0,2] and hail[i][1,2] < hail[j][1,2]):
            for z in range(np.min([hail[i][1,2], hail[j][1,2]]), np.max([hail[i][1,2], hail[j][1,2]])):
                not_z.append(z)
                
    not_x = np.unique(not_x)
    not_y = np.unique(not_y)
    not_z = np.unique(not_z)
    
    max_x = np.max(np.abs(not_x))
    max_y = np.max(np.abs(not_y))
    max_z = np.max(np.abs(not_z))
    
    rng_x = max_x*2
    rng_y = max_y*2
    rng_z = max_z*2
    poss_x = np.setxor1d(np.arange(-1*rng_x, rng_x), not_x)
    poss_y = np.setxor1d(np.arange(-1*rng_y, rng_y), not_y)
    poss_z = np.setxor1d(np.arange(-1*rng_z, rng_z), not_z)
    
    arg_x = np.argsort(np.abs(poss_x))
    arg_y = np.argsort(np.abs(poss_y))
    arg_z = np.argsort(np.abs(poss_z))
    
    return poss_x[arg_x], poss_y[arg_y], poss_z[arg_z]
        

def throw_stone(data):
    hail = get_hail(data)
    
    poss_x, poss_y, poss_z = possible_velocities(hail)
    
    poss_x_y = []
    
    for x in poss_x:
        #print(x)
        for y in poss_y:
            found = True
            target, time = intersect(hail[0], hail[1], [x,y])
            
            if np.isnan(target[0]) or np.isnan(target[1]):
                found = False
                continue
            
            for i in range(2, np.min([10, len(hail)])):
                pos, time = intersect(hail[0], hail[i], [x,y])
                
                if np.isnan(pos[0]) or np.isnan(pos[1]):
                    found = False
                    break
                
                elif pos != target:
                    found = False
                    break
                    
            if found:
                poss_x_y.append([x,y])
            
    for x, y in poss_x_y:
        for z in poss_z:
            found = True
            pos, time = intersect(hail[0], hail[1], [x,y])
            if np.isnan(pos[0]) or np.isnan(pos[1]):
                found = False
                continue

            Pz = hail[0][0,2]+(time[0]*(hail[0][1,2]-z))
            Pz = np.rint(Pz)
            
            target = (pos[0], pos[1], Pz)

            for i in range(2, np.min([3, len(hail)])):
                pos, time = intersect(hail[0], hail[i], [x,y])
                if np.isnan(pos[0]) or np.isnan(pos[1]):
                    found = False
                    break

                Pz = hail[i][0,2]+(time[1]*(hail[i][1,2]-z))
                Pz = np.rint(Pz)
                
                if (pos[0], pos[1], Pz) != target:
                    found = False
                    break

            if found:
                break
        if found:
            break
            
    if found == False:
        print('Failed to find velocity')
        return

    return int(np.sum([pos[0], pos[1], Pz]))

print(throw_stone(test))
print('Part 2 result:', throw_stone(data))

47
Part 2 result: 677656046662770
